# Analytical Finance Assignment 1
### Jordan Ehlinger (je28596)

## Read in and clean data

In [216]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.simplefilter(action="ignore")

In [217]:
funda = pd.read_csv('annual_xrd_expense.csv')
# filter out securities listed on irrelevant exchanges
funda = funda[(funda['exchg'] >= 11) & (funda['exchg'] <= 19)]
# filter out other currencies
funda = funda[funda['curcd'] == 'USD']
# filter out securities not listed in the US
funda = funda[funda['fic'].str.contains('USA')]
# filter out financial companies
funda = funda[~((funda['sic'] >= 6000) & (funda['sic'] <= 6999))]
# filter out pharma companies
funda = funda[~(funda['sic'] == 2834)]
# turn datadate col into datetime
funda['datadate'] = pd.to_datetime(funda['datadate'])
# create new col for mkt cap
funda['mkt_cap'] = funda['prcc_f']*funda['csho']
# convert NaN values to 0 for calculations below
funda['xrd'] = funda['xrd'].fillna(0)
# convert NaN values to 0 for calculations below
funda['mkt_cap'] = funda['mkt_cap'].fillna(0)
# remove rest of un-needed cols
funda = funda[['datadate', 'LPERMNO', 'xrd', 'mkt_cap']]
# create new column for the year
funda['year'] = funda['datadate'].dt.year
# filter out small mkt_cap companies
funda = funda[~(funda['mkt_cap'] < 10)]
# filter instances where there are multiple LPERMNO values for the same year
funda.sort_values (by=['LPERMNO', 'datadate'], ascending=False, inplace=True)
funda = funda.drop_duplicates(subset=['year', 'LPERMNO'])

funda

,datadate,LPERMNO,xrd,mkt_cap,year
302459,2022-12-31,93436,3075.000,3.897415e+05,2022
302458,2021-12-31,93436,2593.000,1.091654e+06,2021
302457,2020-12-31,93436,1491.000,6.774432e+05,2020
302456,2019-12-31,93436,1390.000,7.571773e+04,2019
302455,2018-12-31,93436,1473.634,5.744228e+04,2018
...,...,...,...,...,...
147094,1994-06-30,10001,0.000,1.862350e+01,1994
147093,1993-06-30,10001,0.000,1.798500e+01,1993
147092,1992-06-30,10001,0.000,1.263125e+01,1992
147091,1991-06-30,10001,0.000,1.126650e+01,1991


In [218]:
sec_mon = pd.read_csv('monthly_security_data.csv')
# filter out securities listed on irrelevant exchanges
sec_mon = sec_mon[(sec_mon['exchg'] >= 11) & (sec_mon['exchg'] <= 19)]
# filter out securities not listed in the US
sec_mon = sec_mon[sec_mon['fic'].str.contains('USA')]
# filter out financial companies
sec_mon = sec_mon[~((sec_mon['sic'] >= 6000) & (sec_mon['sic'] <= 6999))]
# filter out pharma companies
sec_mon = sec_mon[~(sec_mon['sic'] == 2834)]
# convert non-numeric values to NaN
sec_mon['trt1m'] = pd.to_numeric(sec_mon['trt1m'], errors='coerce')
# remove rows with NaN in 'trt1m' column
sec_mon = sec_mon.dropna(subset=['trt1m'])
# filter out returns below -100
sec_mon = sec_mon[~(sec_mon['trt1m'] < -100)]
# filter out returns above 1000
sec_mon = sec_mon[sec_mon['trt1m'] < 1000]
# turn datadate col into datetime
sec_mon['datadate'] = pd.to_datetime(sec_mon['datadate'])
# filter out un-needed cols
sec_mon = sec_mon[['datadate', 'LPERMNO', 'trt1m']]
# create new col for month
sec_mon['month'] = sec_mon['datadate'].dt.month
# create new col for year
sec_mon['year'] = sec_mon['datadate'].dt.year

sec_mon

,datadate,LPERMNO,trt1m,month,year
0,1973-01-31,25881,0.0000,1,1973
1,1974-04-30,25881,9.5238,4,1974
2,1973-04-30,25881,-22.8571,4,1973
3,1973-10-31,25881,-16.6667,10,1973
4,1972-05-31,25881,-7.0175,5,1972
...,...,...,...,...,...
3760242,2022-01-31,15642,-12.4561,1,2022
3760243,2021-12-31,15642,-4.2017,12,2021
3760244,2021-11-30,15642,-16.8994,11,2021
3760245,2021-10-31,15642,-8.5568,10,2021


In [219]:
# read in monthly returns for fama french model
ff_df = pd.read_csv('F-F_Research_Data_Factors.csv', dtype={'date':str})
ff_df['year'] = ff_df['date'].str[:4].astype(int)
ff_df['month'] = ff_df['date'].str[-2:].astype(int)
ff_df.drop(['date'], axis=1, inplace=True)

ff_df

,Mkt-RF,SMB,HML,RF,year,month
0,-8.10,2.93,3.13,0.60,1970,1
1,5.13,-2.58,3.93,0.62,1970,2
2,-1.06,-2.32,3.99,0.57,1970,3
3,-11.00,-6.15,6.18,0.50,1970,4
4,-6.92,-4.59,3.33,0.53,1970,5
...,...,...,...,...,...,...
642,3.21,2.08,4.11,0.45,2023,7
643,-2.39,-3.16,-1.06,0.45,2023,8
644,-5.24,-2.51,1.52,0.43,2023,9
645,-3.19,-3.87,0.19,0.47,2023,10


In [116]:
# LPERMNOs per year after filtering
series1 = funda.groupby('year')['LPERMNO'].count()
series2 = sec_mon.groupby('year')['LPERMNO'].count()
df = pd.DataFrame({'Fundamentals': series1, 'Securities Monthly': series2})
# CHECKPOINT 1
df.to_csv('LPERMNOs per year after filtering.csv')

## Create quintiles + non-R&D

In [220]:
# init dataframes to concat to
RDC_df = pd.DataFrame()

# last starting year for RDC calculations will be 2017 (2017+4 = 2021 which will be R&D values for 2022 portfolio)
for start_year in range(1975, 2017+1):
    # init end year and dict
    end_year = start_year + 4
    result_dict = {}

    # define a dictionary for multipliers for each year
    year_multipliers = {start_year: 0.2, start_year+1: 0.4, start_year+2: 0.6, start_year+3: 0.8, end_year: 1}

    # loop through each year in the range
    for year in range(start_year, end_year + 1):
        # Get the multiplier for the current year
        multiplier = year_multipliers.get(year, 1)
        
        # filter the dataframe for the current year
        year_xrd = funda[funda['year'] == year]
        
        # loop through each row in the filtered df
        for index, row in year_xrd.iterrows():
            # filter by security
            lpermno = row['LPERMNO']
            # apply the year's multiplier
            xrd_sum = row['xrd'] * multiplier
            if xrd_sum == np.nan:
                xrd_sum = 0

        # if the LPERMNO is not in the dictionary, initialize its value
            if lpermno not in result_dict:
                result_dict[lpermno] = 0
            # add the calculated xrd_sum to the accumulated sum for this security
            result_dict[lpermno] += xrd_sum

    # filter the DataFrame by the specific year
    mkt_cap_yr = funda.loc[funda['year'] == end_year].copy()
    # get RDC column using result dict created above
    mkt_cap_yr['RDC'] = mkt_cap_yr['LPERMNO'].map(result_dict)
    # calculate RDC_ME
    mkt_cap_yr['RDC_ME'] = np.where( mkt_cap_yr['RDC'] / mkt_cap_yr['mkt_cap'] == np.nan, 0, mkt_cap_yr['RDC'] / mkt_cap_yr['mkt_cap'])
    # concat the data for this year to a df that holds all the years
    RDC_df = pd.concat([RDC_df, mkt_cap_yr])

# only grab necessary columns
RDC_df = RDC_df[['year', 'LPERMNO', 'RDC_ME', 'mkt_cap', 'xrd']]

In [221]:
# split into quintiles based on RDC_ME
securities_ME_port = pd.DataFrame()
# filter through each year
for year in RDC_df['year'].unique():
    # filter to this year
    data = RDC_df[RDC_df['year']==year]
    # look at positive R&D securities
    port_xrd = data[data['RDC_ME']>0]
    # split data into quantiles
    data['flg_portfolio'] = np.where(data['RDC_ME']<=0,0,
        np.where((data['RDC_ME']>0)&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.2)),1,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.2))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.4)),2,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.4))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.6)),3,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.6))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.8)),4,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.8)),5, np.nan))))))
    securities_ME_port = pd.concat([securities_ME_port,data])
# print how many total securities are present in our portfolios (same security can be represented in multiple years but only once per year)
print(len(securities_ME_port))
# shift forward one year to match with correct returns
securities_ME_port['year'] = securities_ME_port['year']+1
# CHECKPOINT 2
securities_ME_port.to_csv('Quintiles before merging with returns.csv')

144230


In [222]:
# get dataframes that just contain one quintile
Zero_RD_df = securities_ME_port[securities_ME_port['flg_portfolio']==0]
Q1 = securities_ME_port[securities_ME_port['flg_portfolio']==1]
Q2 = securities_ME_port[securities_ME_port['flg_portfolio']==2]
Q3 = securities_ME_port[securities_ME_port['flg_portfolio']==3]
Q4 = securities_ME_port[securities_ME_port['flg_portfolio']==4]
Q5 = securities_ME_port[securities_ME_port['flg_portfolio']==5]

In [223]:
Q1

,year,LPERMNO,RDC_ME,mkt_cap,xrd,flg_portfolio
133800,1980,83300,0.011022,26.130000,0.200,1.0
131085,1980,81702,0.011966,35.149923,0.215,1.0
128137,1980,80144,0.011441,81.721000,0.935,1.0
126410,1980,79231,0.006440,125.001000,0.805,1.0
123176,1980,77295,0.002511,63.721500,0.000,1.0
...,...,...,...,...,...,...
9071,2022,10516,0.012512,37850.400000,171.000,1.0
140236,2022,10333,0.014016,235.917750,0.171,1.0
302690,2022,10220,0.009264,4376.040480,11.059,1.0
139529,2022,10200,0.004788,14651.213640,34.274,1.0


In [224]:
Q5

,year,LPERMNO,RDC_ME,mkt_cap,xrd,flg_portfolio
132106,1980,82262,0.206490,71.485241,6.329,5.0
131151,1980,81745,0.256970,19.966495,3.510,5.0
130493,1980,81518,0.170699,51.800000,3.233,5.0
128041,1980,80128,0.209001,22.009500,1.700,5.0
125228,1980,78298,0.246972,47.008500,4.311,5.0
...,...,...,...,...,...,...
299845,2022,12577,0.345825,301.522480,37.944,5.0
302726,2022,12497,2.429352,112.119600,97.524,5.0
153750,2022,11636,0.488138,931.305650,130.821,5.0
153836,2022,11394,0.509313,12.031120,2.209,5.0


In [122]:
# CHECKPOINT 3
# check quintiles with returns merged
quintile_rtns = pd.merge(securities_ME_port, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
# convert NaN values to 0
quintile_rtns['trt1m'] = quintile_rtns['trt1m'].fillna(0)
quintile_rtns.to_csv('Quintiles after merging with returns.csv')

In [123]:
# CHECKPOINT 3 continued
# avg rtns per month
quintile_rtns.groupby(['year', 'month'])['trt1m'].mean()

year  month
1980  1.0       8.785092
      2.0      -1.622596
      3.0     -16.866395
      4.0       5.867531
      5.0       7.782148
                 ...    
2022  8.0      -0.047733
      9.0     -12.036647
      10.0      7.041349
      11.0      0.262741
      12.0     -6.955894
Name: trt1m, Length: 516, dtype: float64

## Merge quintiles with returns

In [225]:
# merge the quintiles with the returns df
Q1_rtns = pd.merge(Q1, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
# convert NaN values to 0
Q1_rtns['trt1m'] = Q1_rtns['trt1m'].fillna(0)
Q2_rtns = pd.merge(Q2, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q2_rtns['trt1m'] = Q2_rtns['trt1m'].fillna(0)
Q3_rtns = pd.merge(Q3, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q3_rtns['trt1m'] = Q3_rtns['trt1m'].fillna(0)
Q4_rtns = pd.merge(Q4, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q4_rtns['trt1m'] = Q4_rtns['trt1m'].fillna(0)
Q5_rtns = pd.merge(Q5, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q5_rtns['trt1m'] = Q5_rtns['trt1m'].fillna(0)
ZeroRD_rtns = pd.merge(Zero_RD_df, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
ZeroRD_rtns['trt1m'] = ZeroRD_rtns['trt1m'].fillna(0)

In [226]:
# drop NaNs and convert back to int
Q1_rtns = Q1_rtns.dropna(subset=['month'])
Q1_rtns['month'] = Q1_rtns['month'].astype(int)

Q2_rtns = Q2_rtns.dropna(subset=['month'])
Q2_rtns['month'] = Q2_rtns['month'].astype(int)

Q3_rtns = Q3_rtns.dropna(subset=['month'])
Q3_rtns['month'] = Q3_rtns['month'].astype(int)

Q4_rtns = Q4_rtns.dropna(subset=['month'])
Q4_rtns['month'] = Q4_rtns['month'].astype(int)

Q5_rtns = Q5_rtns.dropna(subset=['month'])
Q5_rtns['month'] = Q5_rtns['month'].astype(int)

ZeroRD_rtns = ZeroRD_rtns.dropna(subset=['month'])
ZeroRD_rtns['month'] = ZeroRD_rtns['month'].astype(int)

## Create Equal-Weighted Portfolios

In [227]:
# calculate total monthly returns
tot_Q1_rtns = Q1_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q2_rtns = Q2_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q3_rtns = Q3_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q4_rtns = Q4_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q5_rtns = Q5_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_ZeroRD_rtns = ZeroRD_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()

tot_Q1_rtns

,year,month,trt1m
0,1980,1,10.366265
1,1980,2,-0.231756
2,1980,3,-18.839151
3,1980,4,6.069143
4,1980,5,6.231312
...,...,...,...
511,2022,8,-1.852596
512,2022,9,-11.184550
513,2022,10,5.439642
514,2022,11,-0.056066


### Calculate Returns over Three Time Periods

In [228]:
# combine year and month into a dt column
tot_Q1_rtns['date'] = pd.to_datetime(tot_Q1_rtns['year'].astype(str) + '-' + tot_Q1_rtns['month'].astype(str) + '-01')
tot_Q2_rtns['date'] = pd.to_datetime(tot_Q2_rtns['year'].astype(str) + '-' + tot_Q2_rtns['month'].astype(str) + '-01')
tot_Q3_rtns['date'] = pd.to_datetime(tot_Q3_rtns['year'].astype(str) + '-' + tot_Q3_rtns['month'].astype(str) + '-01')
tot_Q4_rtns['date'] = pd.to_datetime(tot_Q4_rtns['year'].astype(str) + '-' + tot_Q4_rtns['month'].astype(str) + '-01')
tot_Q5_rtns['date'] = pd.to_datetime(tot_Q5_rtns['year'].astype(str) + '-' + tot_Q5_rtns['month'].astype(str) + '-01')
tot_ZeroRD_rtns['date'] = pd.to_datetime(tot_ZeroRD_rtns['year'].astype(str) + '-' + tot_ZeroRD_rtns['month'].astype(str) + '-01')

In [229]:
# Full sample period (1981.07 – 2012.12)
start_date = '1981-07-01'
end_date = '2012-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['trt1m'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['trt1m'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['trt1m'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['trt1m'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['trt1m'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['trt1m'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.408 0.879 1.073 1.241 1.858 0.935


Slide Values: 0.495  &nbsp;&nbsp;  0.636  &nbsp;&nbsp; 0.929  &nbsp;&nbsp;  1.232  &nbsp;&nbsp; 1.704  &nbsp;&nbsp; 0.669

In [230]:
# Pre-2000 period (1981.07 – 1999.12)
start_date = '1981-07-01'
end_date = '1999-12-31'

# calculate the average monthly return
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['trt1m'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['trt1m'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['trt1m'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['trt1m'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['trt1m'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['trt1m'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.599 1.002 1.17 1.367 1.908 0.9


Slide Values: 0.598  &nbsp;&nbsp;  0.551  &nbsp;&nbsp; 0.937  &nbsp;&nbsp;  1.251  &nbsp;&nbsp; 1.696  &nbsp;&nbsp; 0.537

In [231]:
# Post-2000 period (2000.01 – 2012.12)
start_date = '2000-01-01'
end_date = '2012-12-31'

# calculate the average monthly return
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['trt1m'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['trt1m'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['trt1m'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['trt1m'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['trt1m'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['trt1m'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.136 0.703 0.935 1.063 1.786 0.983


Slide Values: 0.348  &nbsp;&nbsp;  0.756  &nbsp;&nbsp; 0.917  &nbsp;&nbsp;  1.204  &nbsp;&nbsp; 1.714  &nbsp;&nbsp; 0.858

## Long-Short Portfolio

In [131]:
# Q5 = high R&D      Q1 = low R&D
tot_Q1_rtns = tot_Q1_rtns.rename(columns={"trt1m": "weighted_return"})
tot_Q5_rtns = tot_Q5_rtns.rename(columns={"trt1m": "weighted_return"})

longshort = pd.merge(tot_Q5_rtns, tot_Q1_rtns, on=['year', 'month'], how='left', suffixes=['_high_RD', '_low_RD'])
longshort['long_short'] = longshort['weighted_return_high_RD'] - longshort['weighted_return_low_RD']
longshort['date'] = longshort['date_high_RD']
longshort['month'] = longshort['date'].dt.month
longshort['year'] = longshort['date'].dt.year
longshort = longshort[['date','year','month','long_short']]
longshort

,date,year,month,long_short
0,1980-01-01,1980,1,-1.116189
1,1980-02-01,1980,2,-2.401723
2,1980-03-01,1980,3,1.160141
3,1980-04-01,1980,4,-3.158923
4,1980-05-01,1980,5,0.369201
...,...,...,...,...
511,2022-08-01,2022,8,7.079976
512,2022-09-01,2022,9,-3.634678
513,2022-10-01,2022,10,-4.357747
514,2022-11-01,2022,11,-3.613753


In [132]:
ls_idx = pd.merge(longshort, ff_df, on=['year', 'month'], how='left')
start_date = '1981-07-01'
end_date = '2012-12-31'
ls_idx = ls_idx[(ls_idx['date'] >= start_date) & (ls_idx['date'] <= end_date)]
ls_idx

,date,year,month,long_short,Mkt-RF,SMB,HML,RF
18,1981-07-01,1981,7,-2.737428,-1.54,-2.19,-0.50,1.24
19,1981-08-01,1981,8,-0.386355,-7.04,-1.95,4.76,1.28
20,1981-09-01,1981,9,3.798942,-7.17,-2.65,5.17,1.24
21,1981-10-01,1981,10,-4.766817,4.92,2.23,-4.21,1.21
22,1981-11-01,1981,11,0.774267,3.36,-1.03,1.83,1.07
...,...,...,...,...,...,...,...,...
391,2012-08-01,2012,8,0.038819,2.55,0.47,1.30,0.01
392,2012-09-01,2012,9,-0.055489,2.73,0.50,1.60,0.01
393,2012-10-01,2012,10,-6.443198,-1.76,-1.15,3.59,0.01
394,2012-11-01,2012,11,-1.653131,0.78,0.63,-0.84,0.01


In [133]:
### CAPM alpha
# declare independent and dependent variable from dataframe
X = ls_idx['Mkt-RF']
y = ls_idx['long_short']
# add a constant to the our independent variable to allow for non-zero intercept
X2 = sm.add_constant(X)

# create the OLS regression model and fit it
lin_reg = sm.OLS(y, X2).fit()
# print a summary
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     10.89
Date:                Mon, 12 Feb 2024   Prob (F-statistic):            0.00106
Time:                        19:11:26   Log-Likelihood:                -1120.2
No. Observations:                 378   AIC:                             2244.
Df Residuals:                     376   BIC:                             2252.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3503      0.244      5.545      0.0

In [134]:
### Fama-French 3-factor alpha
X = ls_idx[['Mkt-RF', 'SMB', 'HML']]
y = ls_idx['long_short']
X2 = sm.add_constant(X)
lin_reg = sm.OLS(y, X2).fit()
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     26.02
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           2.61e-15
Time:                        19:11:26   Log-Likelihood:                -1089.7
No. Observations:                 378   AIC:                             2187.
Df Residuals:                     374   BIC:                             2203.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3268      0.228      5.809      0.0

In [135]:
### Sharpe Ratio
sr = ls_idx['long_short'].mean()/ls_idx['long_short'].std()
sr*np.sqrt(12)

1.055354563993182

## Value-Weighted Portfolios

In [136]:
# drop duplicates of years and securities so we can grab just the yearly market cap
yr_mkt_cap = Q1_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
# select only necessary columns for our calculation & merge
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
# find the total market cap for each year
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
# find the ratio of each security's mkt cap to total mkt cap for that year
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
# merge the calculated weights back to the original dataframe on year and LPERMNO
Q1_rtns_all = Q1_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
# create new column for the weighted return
Q1_rtns_all['weighted_return'] = Q1_rtns_all['trt1m'] * Q1_rtns_all['weight']
Q1_rtns_all

,year,LPERMNO,RDC_ME,mkt_cap,xrd,flg_portfolio,month,trt1m,weight,weighted_return
0,1980,83300,0.011022,26.13000,0.200,1.0,4,68.7500,0.000312,0.021447
1,1980,83300,0.011022,26.13000,0.200,1.0,6,8.1818,0.000312,0.002552
2,1980,83300,0.011022,26.13000,0.200,1.0,5,1.8519,0.000312,0.000578
3,1980,83300,0.011022,26.13000,0.200,1.0,2,-2.5641,0.000312,-0.000800
4,1980,83300,0.011022,26.13000,0.200,1.0,3,-14.6842,0.000312,-0.004581
...,...,...,...,...,...,...,...,...,...,...
164190,2022,10026,0.000663,2916.41688,0.619,1.0,11,11.1171,0.000406,0.004516
164191,2022,10026,0.000663,2916.41688,0.619,1.0,8,9.9771,0.000406,0.004053
164192,2022,10026,0.000663,2916.41688,0.619,1.0,4,-3.4816,0.000406,-0.001414
164193,2022,10026,0.000663,2916.41688,0.619,1.0,3,-4.8785,0.000406,-0.001982


In [137]:
# check if weighting is working properly (should equal 1)
Q1_rtns_all[(Q1_rtns_all['year']==1983) & (Q1_rtns_all['month'] == 5)]['weight'].sum()

0.996962461839102

In [138]:
# find value-weighted returns for the rest of the portfolios
yr_mkt_cap = Q2_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q2_rtns_all = Q2_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q2_rtns_all['weighted_return'] = Q2_rtns_all['trt1m'] * Q2_rtns_all['weight']

yr_mkt_cap = Q3_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q3_rtns_all = Q3_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q3_rtns_all['weighted_return'] = Q3_rtns_all['trt1m'] * Q3_rtns_all['weight']

yr_mkt_cap = Q4_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q4_rtns_all = Q4_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q4_rtns_all['weighted_return'] = Q4_rtns_all['trt1m'] * Q4_rtns_all['weight']

yr_mkt_cap = Q5_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q5_rtns_all = Q5_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q5_rtns_all['weighted_return'] = Q5_rtns_all['trt1m'] * Q5_rtns_all['weight']

yr_mkt_cap = ZeroRD_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
ZeroRD_rtns_all = ZeroRD_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
ZeroRD_rtns_all['weighted_return'] = ZeroRD_rtns_all['trt1m'] * ZeroRD_rtns_all['weight']

In [139]:
# calculate total monthly returns
tot_Q1_rtns = Q1_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q2_rtns = Q2_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q3_rtns = Q3_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q4_rtns = Q4_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q5_rtns = Q5_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_ZeroRD_rtns = ZeroRD_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()

tot_Q1_rtns

,year,month,weighted_return
0,1980,1,11.675206
1,1980,2,1.722513
2,1980,3,-15.508051
3,1980,4,5.477653
4,1980,5,4.074750
...,...,...,...
511,2022,8,-3.992962
512,2022,9,-8.600926
513,2022,10,5.261553
514,2022,11,1.000828


### Calculate Returns over Three Time Periods

In [140]:
# combine year and month into a dt column
tot_Q1_rtns['date'] = pd.to_datetime(tot_Q1_rtns['year'].astype(str) + '-' + tot_Q1_rtns['month'].astype(str) + '-01')
tot_Q2_rtns['date'] = pd.to_datetime(tot_Q2_rtns['year'].astype(str) + '-' + tot_Q2_rtns['month'].astype(str) + '-01')
tot_Q3_rtns['date'] = pd.to_datetime(tot_Q3_rtns['year'].astype(str) + '-' + tot_Q3_rtns['month'].astype(str) + '-01')
tot_Q4_rtns['date'] = pd.to_datetime(tot_Q4_rtns['year'].astype(str) + '-' + tot_Q4_rtns['month'].astype(str) + '-01')
tot_Q5_rtns['date'] = pd.to_datetime(tot_Q5_rtns['year'].astype(str) + '-' + tot_Q5_rtns['month'].astype(str) + '-01')
tot_ZeroRD_rtns['date'] = pd.to_datetime(tot_ZeroRD_rtns['year'].astype(str) + '-' + tot_ZeroRD_rtns['month'].astype(str) + '-01')

In [141]:
# Full sample period (1981.07 – 2012.12)
start_date = '1981-07-01'
end_date = '2012-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.794 1.075 1.026 1.239 1.262 0.907


Slide Values: 0.504  &nbsp;&nbsp;  0.627  &nbsp;&nbsp; 0.720  &nbsp;&nbsp;  0.842  &nbsp;&nbsp; 0.959  &nbsp;&nbsp; 0.570

In [142]:
# Pre-2000 period (1981.07 – 1999.12)
start_date = '1981-07-01'
end_date = '1999-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

1.183 1.529 1.408 1.52 1.548 1.245


Slide Values: 0.810  &nbsp;&nbsp;  0.962  &nbsp;&nbsp; 1.034  &nbsp;&nbsp;  1.128  &nbsp;&nbsp; 1.202  &nbsp;&nbsp; 0.804

In [143]:
# Post-2000 period (2000.01 – 2012.12)
start_date = '2000-01-01'
end_date = '2012-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.242 0.428 0.483 0.838 0.854 0.427


Slide Values: 0.068  &nbsp;&nbsp;  0.151  &nbsp;&nbsp; 0.274  &nbsp;&nbsp;  0.435  &nbsp;&nbsp; 0.614  &nbsp;&nbsp; 0.236

In [144]:
# Q5 = high R&D      Q1 = low R&D
longshort = pd.merge(tot_Q5_rtns, tot_Q1_rtns, on=['year', 'month'], how='left', suffixes=['_high_RD', '_low_RD'])
longshort['long_short'] = longshort['weighted_return_high_RD'] - longshort['weighted_return_low_RD']
longshort['date'] = longshort['date_high_RD']
longshort['month'] = longshort['date'].dt.month
longshort['year'] = longshort['date'].dt.year
longshort = longshort[['date','year','month','long_short']]
longshort

,date,year,month,long_short
0,1980-01-01,1980,1,-3.315880
1,1980-02-01,1980,2,-5.917452
2,1980-03-01,1980,3,4.181254
3,1980-04-01,1980,4,-5.483263
4,1980-05-01,1980,5,0.358229
...,...,...,...,...
511,2022-08-01,2022,8,6.547610
512,2022-09-01,2022,9,0.176981
513,2022-10-01,2022,10,8.627664
514,2022-11-01,2022,11,3.529738


In [145]:
ls_idx = pd.merge(longshort, ff_df, on=['year', 'month'], how='left')
start_date = '1981-07-01'
end_date = '2012-12-31'
ls_idx = ls_idx[(ls_idx['date'] >= start_date) & (ls_idx['date'] <= end_date)]
ls_idx

,date,year,month,long_short,Mkt-RF,SMB,HML,RF
18,1981-07-01,1981,7,-14.628404,-1.54,-2.19,-0.50,1.24
19,1981-08-01,1981,8,-0.845318,-7.04,-1.95,4.76,1.28
20,1981-09-01,1981,9,7.144450,-7.17,-2.65,5.17,1.24
21,1981-10-01,1981,10,-11.189724,4.92,2.23,-4.21,1.21
22,1981-11-01,1981,11,-4.075303,3.36,-1.03,1.83,1.07
...,...,...,...,...,...,...,...,...
391,2012-08-01,2012,8,1.111734,2.55,0.47,1.30,0.01
392,2012-09-01,2012,9,0.547932,2.73,0.50,1.60,0.01
393,2012-10-01,2012,10,3.539821,-1.76,-1.15,3.59,0.01
394,2012-11-01,2012,11,3.261703,0.78,0.63,-0.84,0.01


In [146]:
### CAPM alpha
# declare independent and dependent variable from dataframe
X = ls_idx['Mkt-RF']
y = ls_idx['long_short']
# add a constant to the our independent variable to allow for non-zero intercept
X2 = sm.add_constant(X)

# create the OLS regression model and fit it
lin_reg = sm.OLS(y, X2).fit()
# print a summary
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     31.38
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           4.10e-08
Time:                        19:11:26   Log-Likelihood:                -1145.4
No. Observations:                 378   AIC:                             2295.
Df Residuals:                     376   BIC:                             2303.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2867      0.260      1.101      0.2

In [147]:
### Fama-French 3-factor alpha
X = ls_idx[['Mkt-RF', 'SMB', 'HML']]
y = ls_idx['long_short']
X2 = sm.add_constant(X)
lin_reg = sm.OLS(y, X2).fit()
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     38.08
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           1.70e-21
Time:                        19:11:26   Log-Likelihood:                -1110.2
No. Observations:                 378   AIC:                             2228.
Df Residuals:                     374   BIC:                             2244.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0281      0.241     -0.117      0.9

In [148]:
### Sharpe Ratio
sr = ls_idx['long_short'].mean()/ls_idx['long_short'].std()
sr*np.sqrt(12)

0.3101464454381317

## Portfolios excluding top 1000 largest firms

In [149]:
### Find top 1000 firms by market cap each year
# init dict to hold top firms
top_firms = {}

# create copy of fundamentals df
RDC_copy = RDC_df.copy()

# loop through each year in the range
for year in range(1979, 2021 + 1):
    # filter the dataframe for the current year
    year_mask = RDC_copy[RDC_copy['year'] == year]
    # sort by mkt_cap
    sorted_yr = year_mask.sort_values(by=['mkt_cap'], ascending=False)
    # grab the largest 1000 mkt_cap LPERMNOs
    top_LP = sorted_yr['LPERMNO'].head(1000).tolist()
    # add LPs for this year to the dict
    top_firms[year] = top_LP

top_firms

{1979: [12490,
  10401,
  11850,
  12079,
  19553,
  15966,
  12060,
  21450,
  14541,
  10604,
  14736,
  11754,
  13928,
  14322,
  21768,
  18163,
  19414,
  22592,
  11703,
  20626,
  27983,
  11471,
  23819,
  18542,
  13901,
  11308,
  21004,
  14656,
  26542,
  39917,
  14891,
  25785,
  48725,
  14218,
  19561,
  10890,
  21709,
  27828,
  13047,
  12749,
  12570,
  13688,
  23915,
  20482,
  10161,
  40416,
  43916,
  13856,
  35211,
  19350,
  18403,
  24109,
  22103,
  24942,
  15579,
  34833,
  19254,
  24643,
  18374,
  25005,
  19537,
  17830,
  10225,
  43123,
  28484,
  21573,
  14525,
  20853,
  33814,
  25769,
  43449,
  15368,
  18411,
  24715,
  47941,
  14090,
  23341,
  28134,
  12052,
  22779,
  15720,
  18550,
  18948,
  11404,
  36505,
  46682,
  20730,
  27051,
  26825,
  10989,
  10145,
  59328,
  52564,
  11981,
  27959,
  26729,
  39087,
  36679,
  23712,
  17144,
  23317,
  26403,
  27086,
  21397,
  40635,
  48506,
  27190,
  18729,
  26294,
  24272,
  14

In [150]:
### Filter out top 1000 firms from RDC df
# init an empty mask
filter_mask = pd.Series([False] * len(RDC_copy))

# iterate through the dictionary of top firms
for year, lpermno in top_firms.items():
    # update the mask to identify rows to remove
    mask_for_year = (RDC_copy['year'] == year) & (RDC_copy['LPERMNO'].isin(lpermno))
    filter_mask = filter_mask | mask_for_year

# use the inverse of the mask to filter the dataframe
RDC_copy = RDC_copy[~filter_mask]
RDC_copy

,year,LPERMNO,RDC_ME,mkt_cap,xrd
89497,1979,84794,0.000000,39.864750,0.000
135535,1979,84145,0.000000,11.243248,0.000
134879,1979,83783,0.021898,63.321500,0.813
134782,1979,83724,0.000000,22.035000,0.000
134572,1979,83620,0.082605,11.461750,0.375
...,...,...,...,...,...
169466,2021,10051,0.000000,702.519370,0.000
137910,2021,10044,0.000000,32.131460,0.000
146749,2021,10032,0.000000,2507.682270,0.000
138576,2021,10028,0.000000,109.584750,0.000


In [151]:
# split into quintiles based on RDC_ME
securities_ME_port = pd.DataFrame()
for year in RDC_copy['year'].unique():
    data = RDC_copy[RDC_copy['year']==year]
    port_xrd = data[data['RDC_ME']>0]
    data['flg_portfolio'] = np.where(data['RDC_ME']<=0,0,
        np.where((data['RDC_ME']>0)&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.2)),1,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.2))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.4)),2,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.4))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.6)),3,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.6))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.8)),4,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.8)),5, np.nan))))))
    securities_ME_port = pd.concat([securities_ME_port,data])
print(len(securities_ME_port))
# shift forward one year to match returns
securities_ME_port['year'] = securities_ME_port['year']+1
securities_ME_port.head()

101240


,year,LPERMNO,RDC_ME,mkt_cap,xrd,flg_portfolio
89497,1980,84794,0.000000,39.864750,0.000,0.0
135535,1980,84145,0.000000,11.243248,0.000,0.0
134879,1980,83783,0.021898,63.321500,0.813,2.0
134782,1980,83724,0.000000,22.035000,0.000,0.0
134572,1980,83620,0.082605,11.461750,0.375,3.0


In [152]:
# split quintiles into their own dfs
Zero_RD_df = securities_ME_port[securities_ME_port['flg_portfolio']==0]
Q1 = securities_ME_port[securities_ME_port['flg_portfolio']==1]
Q2 = securities_ME_port[securities_ME_port['flg_portfolio']==2]
Q3 = securities_ME_port[securities_ME_port['flg_portfolio']==3]
Q4 = securities_ME_port[securities_ME_port['flg_portfolio']==4]
Q5 = securities_ME_port[securities_ME_port['flg_portfolio']==5]

In [153]:
# merge the quintiles with the returns df
Q1_rtns = pd.merge(Q1, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q2_rtns = pd.merge(Q2, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q3_rtns = pd.merge(Q3, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q4_rtns = pd.merge(Q4, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q5_rtns = pd.merge(Q5, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
ZeroRD_rtns = pd.merge(Zero_RD_df, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')

In [154]:
# drop NaNs and convert back to int
Q1_rtns = Q1_rtns.dropna(subset=['month'])
Q1_rtns['month'] = Q1_rtns['month'].astype(int)

Q2_rtns = Q2_rtns.dropna(subset=['month'])
Q2_rtns['month'] = Q2_rtns['month'].astype(int)

Q3_rtns = Q3_rtns.dropna(subset=['month'])
Q3_rtns['month'] = Q3_rtns['month'].astype(int)

Q4_rtns = Q4_rtns.dropna(subset=['month'])
Q4_rtns['month'] = Q4_rtns['month'].astype(int)

Q5_rtns = Q5_rtns.dropna(subset=['month'])
Q5_rtns['month'] = Q5_rtns['month'].astype(int)

ZeroRD_rtns = ZeroRD_rtns.dropna(subset=['month'])
ZeroRD_rtns['month'] = ZeroRD_rtns['month'].astype(int)

In [155]:
# drop duplicates of years and securities so we can grab just the yearly market cap
yr_mkt_cap = Q1_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
# select only necessary columns for our calculation & merge
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
# find the total market cap for each year
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
# find the ratio of each security's mkt cap to total mkt cap for that year
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
# merge the calculated weights back to the original dataframe on year and LPERMNO
Q1_rtns_all = Q1_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
# create new column for the weighted return
Q1_rtns_all['weighted_return'] = Q1_rtns_all['trt1m'] * Q1_rtns_all['weight']
Q1_rtns_all

,year,LPERMNO,RDC_ME,mkt_cap,xrd,flg_portfolio,month,trt1m,weight,weighted_return
0,1980,83300,0.011022,26.13000,0.200,1.0,4,68.7500,0.007956,0.546952
1,1980,83300,0.011022,26.13000,0.200,1.0,6,8.1818,0.007956,0.065092
2,1980,83300,0.011022,26.13000,0.200,1.0,5,1.8519,0.007956,0.014733
3,1980,83300,0.011022,26.13000,0.200,1.0,2,-2.5641,0.007956,-0.020399
4,1980,83300,0.011022,26.13000,0.200,1.0,3,-14.6842,0.007956,-0.116822
...,...,...,...,...,...,...,...,...,...,...
116841,2022,10026,0.000663,2916.41688,0.619,1.0,11,11.1171,0.009129,0.101483
116842,2022,10026,0.000663,2916.41688,0.619,1.0,8,9.9771,0.009129,0.091077
116843,2022,10026,0.000663,2916.41688,0.619,1.0,4,-3.4816,0.009129,-0.031782
116844,2022,10026,0.000663,2916.41688,0.619,1.0,3,-4.8785,0.009129,-0.044534


In [156]:
# find value-weighted returns for the rest of the portfolios
yr_mkt_cap = Q2_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q2_rtns_all = Q2_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q2_rtns_all['weighted_return'] = Q2_rtns_all['trt1m'] * Q2_rtns_all['weight']

yr_mkt_cap = Q3_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q3_rtns_all = Q3_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q3_rtns_all['weighted_return'] = Q3_rtns_all['trt1m'] * Q3_rtns_all['weight']

yr_mkt_cap = Q4_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q4_rtns_all = Q4_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q4_rtns_all['weighted_return'] = Q4_rtns_all['trt1m'] * Q4_rtns_all['weight']

yr_mkt_cap = Q5_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q5_rtns_all = Q5_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q5_rtns_all['weighted_return'] = Q5_rtns_all['trt1m'] * Q5_rtns_all['weight']

yr_mkt_cap = ZeroRD_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
ZeroRD_rtns_all = ZeroRD_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
ZeroRD_rtns_all['weighted_return'] = ZeroRD_rtns_all['trt1m'] * ZeroRD_rtns_all['weight']

In [157]:
# calculate total monthly returns
tot_Q1_rtns = Q1_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q2_rtns = Q2_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q3_rtns = Q3_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q4_rtns = Q4_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q5_rtns = Q5_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_ZeroRD_rtns = ZeroRD_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()

tot_Q1_rtns

,year,month,weighted_return
0,1980,1,11.124756
1,1980,2,-0.809626
2,1980,3,-20.678422
3,1980,4,7.441306
4,1980,5,5.884915
...,...,...,...
511,2022,8,-3.109023
512,2022,9,-11.062297
513,2022,10,9.945297
514,2022,11,-3.447407


### Calculate Returns over Three Time Periods

In [158]:
# combine year and month into a dt column
tot_Q1_rtns['date'] = pd.to_datetime(tot_Q1_rtns['year'].astype(str) + '-' + tot_Q1_rtns['month'].astype(str) + '-01')
tot_Q2_rtns['date'] = pd.to_datetime(tot_Q2_rtns['year'].astype(str) + '-' + tot_Q2_rtns['month'].astype(str) + '-01')
tot_Q3_rtns['date'] = pd.to_datetime(tot_Q3_rtns['year'].astype(str) + '-' + tot_Q3_rtns['month'].astype(str) + '-01')
tot_Q4_rtns['date'] = pd.to_datetime(tot_Q4_rtns['year'].astype(str) + '-' + tot_Q4_rtns['month'].astype(str) + '-01')
tot_Q5_rtns['date'] = pd.to_datetime(tot_Q5_rtns['year'].astype(str) + '-' + tot_Q5_rtns['month'].astype(str) + '-01')
tot_ZeroRD_rtns['date'] = pd.to_datetime(tot_ZeroRD_rtns['year'].astype(str) + '-' + tot_ZeroRD_rtns['month'].astype(str) + '-01')

In [159]:
# Full sample period (1981.07 – 2012.12)
start_date = '1981-07-01'
end_date = '2012-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.396 0.863 0.976 1.274 1.701 0.867


Slide Values: 0.260  &nbsp;&nbsp;  0.525  &nbsp;&nbsp; 0.849  &nbsp;&nbsp;  0.963  &nbsp;&nbsp; 1.363  &nbsp;&nbsp; 0.588

In [160]:
# Pre-2000 period (1981.07 – 1999.12)
start_date = '1981-07-01'
end_date = '1999-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.519 0.918 1.064 1.338 1.666 0.822


Slide Values: 0.188  &nbsp;&nbsp;  0.406  &nbsp;&nbsp; 0.878  &nbsp;&nbsp;  0.981  &nbsp;&nbsp; 1.317  &nbsp;&nbsp; 0.452

In [161]:
# Post-2000 period (2000.01 – 2012.12)
start_date = '2000-01-01'
end_date = '2012-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.22 0.785 0.85 1.182 1.751 0.932


Slide Values: 0.363  &nbsp;&nbsp;  0.694  &nbsp;&nbsp; 0.807  &nbsp;&nbsp;  0.937  &nbsp;&nbsp; 1.428  &nbsp;&nbsp; 0.783

In [162]:
# Q5 = high R&D      Q1 = low R&D
longshort = pd.merge(tot_Q5_rtns, tot_Q1_rtns, on=['year', 'month'], how='left', suffixes=['_high_RD', '_low_RD'])
longshort['long_short'] = longshort['weighted_return_high_RD'] - longshort['weighted_return_low_RD']
longshort['date'] = longshort['date_high_RD']
longshort['month'] = longshort['date'].dt.month
longshort['year'] = longshort['date'].dt.year
longshort = longshort[['date','year','month','long_short']]
longshort

,date,year,month,long_short
0,1980-01-01,1980,1,-2.755473
1,1980-02-01,1980,2,-1.618327
2,1980-03-01,1980,3,3.410739
3,1980-04-01,1980,4,-4.846813
4,1980-05-01,1980,5,1.739044
...,...,...,...,...
511,2022-08-01,2022,8,15.068120
512,2022-09-01,2022,9,-1.034403
513,2022-10-01,2022,10,-4.317110
514,2022-11-01,2022,11,1.329962


In [163]:
ls_idx = pd.merge(longshort, ff_df, on=['year', 'month'], how='left')
start_date = '1981-07-01'
end_date = '2012-12-31'
ls_idx = ls_idx[(ls_idx['date'] >= start_date) & (ls_idx['date'] <= end_date)]
ls_idx

,date,year,month,long_short,Mkt-RF,SMB,HML,RF
18,1981-07-01,1981,7,0.497682,-1.54,-2.19,-0.50,1.24
19,1981-08-01,1981,8,2.794515,-7.04,-1.95,4.76,1.28
20,1981-09-01,1981,9,4.272547,-7.17,-2.65,5.17,1.24
21,1981-10-01,1981,10,-1.678006,4.92,2.23,-4.21,1.21
22,1981-11-01,1981,11,1.867983,3.36,-1.03,1.83,1.07
...,...,...,...,...,...,...,...,...
391,2012-08-01,2012,8,0.107879,2.55,0.47,1.30,0.01
392,2012-09-01,2012,9,-0.586490,2.73,0.50,1.60,0.01
393,2012-10-01,2012,10,-6.239688,-1.76,-1.15,3.59,0.01
394,2012-11-01,2012,11,-0.125375,0.78,0.63,-0.84,0.01


In [164]:
### CAPM alpha
# declare independent and dependent variable from dataframe
X = ls_idx['Mkt-RF']
y = ls_idx['long_short']
# add a constant to the our independent variable to allow for non-zero intercept
X2 = sm.add_constant(X)

# create the OLS regression model and fit it
lin_reg = sm.OLS(y, X2).fit()
# print a summary
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     11.06
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           0.000966
Time:                        19:11:29   Log-Likelihood:                -1110.7
No. Observations:                 378   AIC:                             2225.
Df Residuals:                     376   BIC:                             2233.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2076      0.237      5.085      0.0

In [165]:
### Fama-French 3-factor alpha
X = ls_idx[['Mkt-RF', 'SMB', 'HML']]
y = ls_idx['long_short']
X2 = sm.add_constant(X)
lin_reg = sm.OLS(y, X2).fit()
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     12.58
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           7.49e-08
Time:                        19:11:29   Log-Likelihood:                -1098.0
No. Observations:                 378   AIC:                             2204.
Df Residuals:                     374   BIC:                             2220.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1772      0.233      5.042      0.0

In [166]:
### Sharpe Ratio
sr = ls_idx['long_short'].mean()/ls_idx['long_short'].std()
sr*np.sqrt(12)

0.9741051391737053

## Extend sample period

### Step 1:

In [167]:
# init dataframes to concat to
RDC_df = pd.DataFrame()

# last starting year for RDC calculations will be 2017 (2017+4 = 2021 which will be R&D values for 2022 portfolio)
for start_year in range(1975, 2017+1):
    # init end year and dict
    end_year = start_year + 4
    result_dict = {}

    # define a dictionary for multipliers for each year
    year_multipliers = {start_year: 0.2, start_year+1: 0.4, start_year+2: 0.6, start_year+3: 0.8, end_year: 1}

    # loop through each year in the range
    for year in range(start_year, end_year + 1):
        # Get the multiplier for the current year
        multiplier = year_multipliers.get(year, 1)
        
        # filter the dataframe for the current year
        year_xrd = funda[funda['year'] == year]
        
        # loop through each row in the filtered df
        for index, row in year_xrd.iterrows():
            # filter by security
            lpermno = row['LPERMNO']
            # apply the year's multiplier
            xrd_sum = row['xrd'] * multiplier
            if xrd_sum == np.nan:
                xrd_sum = 0

        # if the LPERMNO is not in the dictionary, initialize its value
            if lpermno not in result_dict:
                result_dict[lpermno] = 0
            # add the calculated xrd_sum to the accumulated sum for this security
            result_dict[lpermno] += xrd_sum

    # filter the DataFrame by the specific year and push forward one year so it matches the return years
    mkt_cap_yr = funda.loc[funda['year'] == end_year].copy()
    mkt_cap_yr['RDC'] = mkt_cap_yr['LPERMNO'].map(result_dict)
    mkt_cap_yr['RDC_ME'] = np.where( mkt_cap_yr['RDC'] / mkt_cap_yr['mkt_cap'] == np.nan, 0, mkt_cap_yr['RDC'] / mkt_cap_yr['mkt_cap'])
    RDC_df = pd.concat([RDC_df, mkt_cap_yr])

In [168]:
securities_ME_port = pd.DataFrame()
for year in RDC_df['year'].unique():
    data = RDC_df[RDC_df['year']==year]
    port_xrd = data[data['RDC_ME']>0]
    data['flg_portfolio'] = np.where(data['RDC_ME']<=0,0,
        np.where((data['RDC_ME']>0)&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.2)),1,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.2))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.4)),2,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.4))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.6)),3,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.6))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.8)),4,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.8)),5, np.nan))))))
    securities_ME_port = pd.concat([securities_ME_port,data])
print(len(securities_ME_port))
securities_ME_port['year'] = securities_ME_port['year']+1
securities_ME_port.head()

144230


,datadate,LPERMNO,xrd,mkt_cap,year,RDC,RDC_ME,flg_portfolio
89497,1979-09-30,84794,0.000,39.864750,1980,0.0000,0.000000,0.0
135535,1979-12-31,84145,0.000,11.243248,1980,0.0000,0.000000,0.0
134879,1979-03-31,83783,0.813,63.321500,1980,1.3866,0.021898,2.0
134782,1979-01-31,83724,0.000,22.035000,1980,0.0000,0.000000,0.0
134706,1979-12-31,83687,3.284,243.729000,1980,7.8534,0.032222,2.0


In [169]:
Zero_RD_df = securities_ME_port[securities_ME_port['flg_portfolio']==0]
Q1 = securities_ME_port[securities_ME_port['flg_portfolio']==1]
Q2 = securities_ME_port[securities_ME_port['flg_portfolio']==2]
Q3 = securities_ME_port[securities_ME_port['flg_portfolio']==3]
Q4 = securities_ME_port[securities_ME_port['flg_portfolio']==4]
Q5 = securities_ME_port[securities_ME_port['flg_portfolio']==5]

In [170]:
# merge the quintiles with the returns df
Q1_rtns = pd.merge(Q1, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q2_rtns = pd.merge(Q2, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q3_rtns = pd.merge(Q3, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q4_rtns = pd.merge(Q4, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q5_rtns = pd.merge(Q5, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
ZeroRD_rtns = pd.merge(Zero_RD_df, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')

In [171]:
# drop NaNs and convert back to int; this may cause a mismatch in the future but I think it is okay because 
# all the NaNs should be from the first three months of 1980 which we don't want to count
Q1_rtns = Q1_rtns.dropna(subset=['month'])
Q1_rtns['month'] = Q1_rtns['month'].astype(int)

Q2_rtns = Q2_rtns.dropna(subset=['month'])
Q2_rtns['month'] = Q2_rtns['month'].astype(int)

Q3_rtns = Q3_rtns.dropna(subset=['month'])
Q3_rtns['month'] = Q3_rtns['month'].astype(int)

Q4_rtns = Q4_rtns.dropna(subset=['month'])
Q4_rtns['month'] = Q4_rtns['month'].astype(int)

Q5_rtns = Q5_rtns.dropna(subset=['month'])
Q5_rtns['month'] = Q5_rtns['month'].astype(int)

ZeroRD_rtns = ZeroRD_rtns.dropna(subset=['month'])
ZeroRD_rtns['month'] = ZeroRD_rtns['month'].astype(int)

In [172]:
# calculate total monthly returns
tot_Q1_rtns = Q1_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q2_rtns = Q2_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q3_rtns = Q3_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q4_rtns = Q4_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_Q5_rtns = Q5_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()
tot_ZeroRD_rtns = ZeroRD_rtns.groupby(['year', 'month'])['trt1m'].mean().reset_index()

#### Calculate Returns over Extended Time Periods

In [173]:
# combine year and month into a dt column
tot_Q1_rtns['date'] = pd.to_datetime(tot_Q1_rtns['year'].astype(str) + '-' + tot_Q1_rtns['month'].astype(str) + '-01')
tot_Q2_rtns['date'] = pd.to_datetime(tot_Q2_rtns['year'].astype(str) + '-' + tot_Q2_rtns['month'].astype(str) + '-01')
tot_Q3_rtns['date'] = pd.to_datetime(tot_Q3_rtns['year'].astype(str) + '-' + tot_Q3_rtns['month'].astype(str) + '-01')
tot_Q4_rtns['date'] = pd.to_datetime(tot_Q4_rtns['year'].astype(str) + '-' + tot_Q4_rtns['month'].astype(str) + '-01')
tot_Q5_rtns['date'] = pd.to_datetime(tot_Q5_rtns['year'].astype(str) + '-' + tot_Q5_rtns['month'].astype(str) + '-01')
tot_ZeroRD_rtns['date'] = pd.to_datetime(tot_ZeroRD_rtns['year'].astype(str) + '-' + tot_ZeroRD_rtns['month'].astype(str) + '-01')

In [174]:
# Full sample period with extension (1981.07 – 2022.12)
start_date = '1981-07-01'
end_date = '2022-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['trt1m'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['trt1m'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['trt1m'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['trt1m'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['trt1m'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['trt1m'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.424 0.86 1.081 1.185 1.681 0.905


In [175]:
# Extended period (2013.01 – 2022.12)
start_date = '2013-01-01'
end_date = '2022-12-31'

# calculate the average monthly return
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['trt1m'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['trt1m'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['trt1m'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['trt1m'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['trt1m'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['trt1m'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.475 0.801 1.106 1.01 1.123 0.812


In [176]:
### grab dataframe for final df
start_date = '1981-07-01'
end_date = '2022-12-31'

tot_Q1_rtns = tot_Q1_rtns.rename(columns={"trt1m": "weighted_return"})
tot_Q2_rtns = tot_Q2_rtns.rename(columns={"trt1m": "weighted_return"})
tot_Q3_rtns = tot_Q3_rtns.rename(columns={"trt1m": "weighted_return"})
tot_Q4_rtns = tot_Q4_rtns.rename(columns={"trt1m": "weighted_return"})
tot_Q5_rtns = tot_Q5_rtns.rename(columns={"trt1m": "weighted_return"})
tot_ZeroRD_rtns = tot_ZeroRD_rtns.rename(columns={"trt1m": "weighted_return"})

# filter the returns to the sample period
Q1_rtns_df = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]
Q2_rtns_df = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]
Q3_rtns_df = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]
Q4_rtns_df = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]
Q5_rtns_df = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]
ZeroRD_rtns_df = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]

# merge all the returns into one df
step1df = pd.merge(Q1_rtns_df[['weighted_return', 'date']], Q2_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q2'])
step1df = pd.merge(step1df, Q3_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q3'])
step1df = pd.merge(step1df, Q4_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q4'])
step1df = pd.merge(step1df, Q5_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q5'])
step1df = pd.merge(step1df, ZeroRD_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_NoRD'])
# rename then drop the Q1 weighted return
step1df['weighted_return_Q1'] = step1df['weighted_return']
step1df.drop('weighted_return', axis=1, inplace=True)
step1df

,date,weighted_return_Q2,weighted_return_Q3,weighted_return_Q4,weighted_return_Q5,weighted_return_NoRD,weighted_return_Q1
0,1981-07-01,-2.799800,-3.457001,-3.606291,-4.290787,-1.309947,-1.553359
1,1981-08-01,-8.570050,-8.670321,-10.652510,-9.499001,-6.909227,-9.112646
2,1981-09-01,-9.134037,-6.536604,-7.867166,-7.552083,-8.629938,-11.351025
3,1981-10-01,9.297802,8.675598,7.015914,6.444521,7.552285,11.211338
4,1981-11-01,0.127447,4.699210,1.494010,2.537728,2.295251,1.763461
...,...,...,...,...,...,...,...
493,2022-08-01,-3.811503,-1.377376,1.296049,5.227380,0.022568,-1.852596
494,2022-09-01,-11.264344,-11.418521,-10.947752,-14.819228,-12.201691,-11.184550
495,2022-10-01,8.146449,5.740109,4.810552,1.081895,10.038750,5.439642
496,2022-11-01,-0.568152,0.196352,-0.152652,-3.669819,1.933884,-0.056066


### Step 2:

In [177]:
# Q5 = high R&D      Q1 = low R&D
longshort = pd.merge(tot_Q5_rtns, tot_Q1_rtns, on=['year', 'month'], how='left', suffixes=['_high_RD', '_low_RD'])
longshort['long_short'] = longshort['weighted_return_high_RD'] - longshort['weighted_return_low_RD']
longshort['date'] = longshort['date_high_RD']
longshort['month'] = longshort['date'].dt.month
longshort['year'] = longshort['date'].dt.year
longshort = longshort[['date','year','month','long_short']]
longshort

,date,year,month,long_short
0,1980-01-01,1980,1,-1.116189
1,1980-02-01,1980,2,-2.401723
2,1980-03-01,1980,3,1.160141
3,1980-04-01,1980,4,-3.158923
4,1980-05-01,1980,5,0.369201
...,...,...,...,...
511,2022-08-01,2022,8,7.079976
512,2022-09-01,2022,9,-3.634678
513,2022-10-01,2022,10,-4.357747
514,2022-11-01,2022,11,-3.613753


In [178]:
ls_idx = pd.merge(longshort, ff_df, on=['year', 'month'], how='left')
start_date = '1981-07-01'
end_date = '2022-12-31'
ls_idx = ls_idx[(ls_idx['date'] >= start_date) & (ls_idx['date'] <= end_date)]
ls_idx

,date,year,month,long_short,Mkt-RF,SMB,HML,RF
18,1981-07-01,1981,7,-2.737428,-1.54,-2.19,-0.50,1.24
19,1981-08-01,1981,8,-0.386355,-7.04,-1.95,4.76,1.28
20,1981-09-01,1981,9,3.798942,-7.17,-2.65,5.17,1.24
21,1981-10-01,1981,10,-4.766817,4.92,2.23,-4.21,1.21
22,1981-11-01,1981,11,0.774267,3.36,-1.03,1.83,1.07
...,...,...,...,...,...,...,...,...
511,2022-08-01,2022,8,7.079976,-3.77,1.40,0.29,0.19
512,2022-09-01,2022,9,-3.634678,-9.35,-0.81,0.05,0.19
513,2022-10-01,2022,10,-4.357747,7.83,0.06,8.01,0.23
514,2022-11-01,2022,11,-3.613753,4.60,-3.52,1.38,0.29


In [179]:
### CAPM alpha
# declare independent and dependent variable from dataframe
X = ls_idx['Mkt-RF']
y = ls_idx['long_short']
# add a constant to the our independent variable to allow for non-zero intercept
X2 = sm.add_constant(X)

# create the OLS regression model and fit it
lin_reg = sm.OLS(y, X2).fit()
# print a summary
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     15.29
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           0.000105
Time:                        19:11:43   Log-Likelihood:                -1471.6
No. Observations:                 498   AIC:                             2947.
Df Residuals:                     496   BIC:                             2956.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1352      0.211      5.382      0.0

In [180]:
### Fama-French 3-factor alpha
X = ls_idx[['Mkt-RF', 'SMB', 'HML']]
y = ls_idx['long_short']
X2 = sm.add_constant(X)
lin_reg = sm.OLS(y, X2).fit()
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     35.88
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           5.44e-21
Time:                        19:11:43   Log-Likelihood:                -1430.1
No. Observations:                 498   AIC:                             2868.
Df Residuals:                     494   BIC:                             2885.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1726      0.196      5.991      0.0

In [181]:
### Sharpe Ratio
sr = ls_idx['long_short'].mean()/ls_idx['long_short'].std()
sr*np.sqrt(12)

0.9220285252540305

In [182]:
### grab dataframe for final df
step2df = pd.merge(step1df, ls_idx[['long_short', 'date']], on='date', how='left')
step2df

,date,weighted_return_Q2,weighted_return_Q3,weighted_return_Q4,weighted_return_Q5,weighted_return_NoRD,weighted_return_Q1,long_short
0,1981-07-01,-2.799800,-3.457001,-3.606291,-4.290787,-1.309947,-1.553359,-2.737428
1,1981-08-01,-8.570050,-8.670321,-10.652510,-9.499001,-6.909227,-9.112646,-0.386355
2,1981-09-01,-9.134037,-6.536604,-7.867166,-7.552083,-8.629938,-11.351025,3.798942
3,1981-10-01,9.297802,8.675598,7.015914,6.444521,7.552285,11.211338,-4.766817
4,1981-11-01,0.127447,4.699210,1.494010,2.537728,2.295251,1.763461,0.774267
...,...,...,...,...,...,...,...,...
493,2022-08-01,-3.811503,-1.377376,1.296049,5.227380,0.022568,-1.852596,7.079976
494,2022-09-01,-11.264344,-11.418521,-10.947752,-14.819228,-12.201691,-11.184550,-3.634678
495,2022-10-01,8.146449,5.740109,4.810552,1.081895,10.038750,5.439642,-4.357747
496,2022-11-01,-0.568152,0.196352,-0.152652,-3.669819,1.933884,-0.056066,-3.613753


### Step 3:

In [183]:
# drop duplicates of years and securities so we can grab just the yearly market cap
yr_mkt_cap = Q1_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
# select only necessary columns for our calculation & merge
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
# find the total market cap for each year
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
# find the ratio of each security's mkt cap to total mkt cap for that year
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
# merge the calculated weights back to the original dataframe on year and LPERMNO
Q1_rtns_all = Q1_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
# create new column for the weighted return
Q1_rtns_all['weighted_return'] = Q1_rtns_all['trt1m'] * Q1_rtns_all['weight']
Q1_rtns_all

,datadate,LPERMNO,xrd,mkt_cap,year,RDC,RDC_ME,flg_portfolio,month,trt1m,weight,weighted_return
0,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,4,68.7500,0.000312,0.021447
1,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,6,8.1818,0.000312,0.002552
2,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,5,1.8519,0.000312,0.000578
3,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,2,-2.5641,0.000312,-0.000800
4,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,3,-14.6842,0.000312,-0.004581
...,...,...,...,...,...,...,...,...,...,...,...,...
164190,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,11,11.1171,0.000406,0.004516
164191,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,8,9.9771,0.000406,0.004053
164192,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,4,-3.4816,0.000406,-0.001414
164193,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,3,-4.8785,0.000406,-0.001982


In [184]:
# find value-weighted returns for the rest of the portfolios
yr_mkt_cap = Q2_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q2_rtns_all = Q2_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q2_rtns_all['weighted_return'] = Q2_rtns_all['trt1m'] * Q2_rtns_all['weight']

yr_mkt_cap = Q3_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q3_rtns_all = Q3_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q3_rtns_all['weighted_return'] = Q3_rtns_all['trt1m'] * Q3_rtns_all['weight']

yr_mkt_cap = Q4_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q4_rtns_all = Q4_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q4_rtns_all['weighted_return'] = Q4_rtns_all['trt1m'] * Q4_rtns_all['weight']

yr_mkt_cap = Q5_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q5_rtns_all = Q5_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q5_rtns_all['weighted_return'] = Q5_rtns_all['trt1m'] * Q5_rtns_all['weight']

yr_mkt_cap = ZeroRD_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
ZeroRD_rtns_all = ZeroRD_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
ZeroRD_rtns_all['weighted_return'] = ZeroRD_rtns_all['trt1m'] * ZeroRD_rtns_all['weight']

In [185]:
# calculate total monthly returns
tot_Q1_rtns = Q1_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q2_rtns = Q2_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q3_rtns = Q3_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q4_rtns = Q4_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q5_rtns = Q5_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_ZeroRD_rtns = ZeroRD_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()

tot_Q1_rtns

,year,month,weighted_return
0,1980,1,11.675206
1,1980,2,1.722513
2,1980,3,-15.508051
3,1980,4,5.477653
4,1980,5,4.074750
...,...,...,...
511,2022,8,-3.992962
512,2022,9,-8.600926
513,2022,10,5.261553
514,2022,11,1.000828


In [186]:
# combine year and month into a dt column
tot_Q1_rtns['date'] = pd.to_datetime(tot_Q1_rtns['year'].astype(str) + '-' + tot_Q1_rtns['month'].astype(str) + '-01')
tot_Q2_rtns['date'] = pd.to_datetime(tot_Q2_rtns['year'].astype(str) + '-' + tot_Q2_rtns['month'].astype(str) + '-01')
tot_Q3_rtns['date'] = pd.to_datetime(tot_Q3_rtns['year'].astype(str) + '-' + tot_Q3_rtns['month'].astype(str) + '-01')
tot_Q4_rtns['date'] = pd.to_datetime(tot_Q4_rtns['year'].astype(str) + '-' + tot_Q4_rtns['month'].astype(str) + '-01')
tot_Q5_rtns['date'] = pd.to_datetime(tot_Q5_rtns['year'].astype(str) + '-' + tot_Q5_rtns['month'].astype(str) + '-01')
tot_ZeroRD_rtns['date'] = pd.to_datetime(tot_ZeroRD_rtns['year'].astype(str) + '-' + tot_ZeroRD_rtns['month'].astype(str) + '-01')

In [187]:
# Full sample period with Extended Period (1981.07 – 2022.12)
start_date = '1981-07-01'
end_date = '2022-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.76 1.126 1.088 1.267 1.296 0.903


In [188]:
# Extended period (2013.01 – 2022.12)
start_date = '2013-01-01'
end_date = '2022-12-31'

# calculate the average monthly return
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.653 1.29 1.28 1.357 1.405 0.889


In [189]:
# Q5 = high R&D      Q1 = low R&D
longshort = pd.merge(tot_Q5_rtns, tot_Q1_rtns, on=['year', 'month'], how='left', suffixes=['_high_RD', '_low_RD'])
longshort['long_short'] = longshort['weighted_return_high_RD'] - longshort['weighted_return_low_RD']
longshort['date'] = longshort['date_high_RD']
longshort['month'] = longshort['date'].dt.month
longshort['year'] = longshort['date'].dt.year
longshort = longshort[['date','year','month','long_short']]

In [190]:
ls_idx = pd.merge(longshort, ff_df, on=['year', 'month'], how='left')
start_date = '1981-07-01'
end_date = '2022-12-31'
ls_idx = ls_idx[(ls_idx['date'] >= start_date) & (ls_idx['date'] <= end_date)]
ls_idx

,date,year,month,long_short,Mkt-RF,SMB,HML,RF
18,1981-07-01,1981,7,-14.628404,-1.54,-2.19,-0.50,1.24
19,1981-08-01,1981,8,-0.845318,-7.04,-1.95,4.76,1.28
20,1981-09-01,1981,9,7.144450,-7.17,-2.65,5.17,1.24
21,1981-10-01,1981,10,-11.189724,4.92,2.23,-4.21,1.21
22,1981-11-01,1981,11,-4.075303,3.36,-1.03,1.83,1.07
...,...,...,...,...,...,...,...,...
511,2022-08-01,2022,8,6.547610,-3.77,1.40,0.29,0.19
512,2022-09-01,2022,9,0.176981,-9.35,-0.81,0.05,0.19
513,2022-10-01,2022,10,8.627664,7.83,0.06,8.01,0.23
514,2022-11-01,2022,11,3.529738,4.60,-3.52,1.38,0.29


In [191]:
### CAPM alpha
# declare independent and dependent variable from dataframe
X = ls_idx['Mkt-RF']
y = ls_idx['long_short']
# add a constant to the our independent variable to allow for non-zero intercept
X2 = sm.add_constant(X)

# create the OLS regression model and fit it
lin_reg = sm.OLS(y, X2).fit()
# print a summary
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     42.29
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           1.92e-10
Time:                        19:11:43   Log-Likelihood:                -1489.1
No. Observations:                 498   AIC:                             2982.
Df Residuals:                     496   BIC:                             2991.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3266      0.218      1.495      0.1

In [192]:
### Fama-French 3-factor alpha
X = ls_idx[['Mkt-RF', 'SMB', 'HML']]
y = ls_idx['long_short']
X2 = sm.add_constant(X)
lin_reg = sm.OLS(y, X2).fit()
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     52.70
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           1.44e-29
Time:                        19:11:43   Log-Likelihood:                -1440.3
No. Observations:                 498   AIC:                             2889.
Df Residuals:                     494   BIC:                             2906.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1485      0.200      0.743      0.4

In [193]:
### Sharpe Ratio
sr = ls_idx['long_short'].mean()/ls_idx['long_short'].std()
sr*np.sqrt(12)

0.370042323905812

In [194]:
### grab dataframe for final df
start_date = '1981-07-01'
end_date = '2022-12-31'

# filter the returns to the sample period
Q1_rtns_df = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]
Q2_rtns_df = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]
Q3_rtns_df = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]
Q4_rtns_df = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]
Q5_rtns_df = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]
ZeroRD_rtns_df = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]

# merge all the returns into one df
step3df = pd.merge(Q1_rtns_df[['weighted_return', 'date']], Q2_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q2'])
step3df = pd.merge(step3df, Q3_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q3'])
step3df = pd.merge(step3df, Q4_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q4'])
step3df = pd.merge(step3df, Q5_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q5'])
step3df = pd.merge(step3df, ZeroRD_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_NoRD'])
step3df = pd.merge(step3df, ls_idx[['long_short', 'date']], on='date', how='left')
# rename then drop the Q1 weighted return
step3df['weighted_return_Q1'] = step3df['weighted_return']
step3df.drop('weighted_return', axis=1, inplace=True)
step3df

,date,weighted_return_Q2,weighted_return_Q3,weighted_return_Q4,weighted_return_Q5,weighted_return_NoRD,long_short,weighted_return_Q1
0,1981-07-01,3.212585,-2.897262,-2.702029,-6.210589,1.066085,-14.628404,8.417815
1,1981-08-01,-3.777345,-7.103646,-5.800542,-8.290917,-5.588584,-0.845318,-7.445599
2,1981-09-01,-6.033636,-6.255274,-3.118583,-4.188068,-6.989747,7.144450,-11.332518
3,1981-10-01,6.206295,6.414784,1.514483,-1.413199,7.505173,-11.189724,9.776525
4,1981-11-01,4.728657,4.806934,4.607864,2.695932,4.037957,-4.075303,6.771236
...,...,...,...,...,...,...,...,...
493,2022-08-01,-5.610173,-2.828746,-4.237212,2.554648,-2.232595,6.547610,-3.992962
494,2022-09-01,-11.104407,-11.546655,-9.803153,-8.423946,-9.452919,0.176981,-8.600926
495,2022-10-01,5.511529,-0.683491,10.262508,13.889217,8.841528,8.627664,5.261553
496,2022-11-01,5.510540,7.149928,6.309871,4.530567,6.882075,3.529738,1.000828


### Step 4:

In [195]:
### Find top 1000 firms by market cap each year
# init dict to hold top firms
top_firms = {}

# create copy of fundamentals df
RDC_copy = RDC_df.copy()

# loop through each year in the range
for year in range(1979, 2021 + 1):
    # filter the dataframe for the current year
    year_mask = RDC_copy[RDC_copy['year'] == year]
    # sort by mkt_cap
    sorted_yr = year_mask.sort_values(by=['mkt_cap'], ascending=False)
    # grab the largest 1000 mkt_cap LPERMNOs
    top_LP = sorted_yr['LPERMNO'].head(1000).tolist()
    # add LPs for this year to the dict
    top_firms[year] = top_LP

top_firms

{1979: [12490,
  10401,
  11850,
  12079,
  19553,
  15966,
  12060,
  21450,
  14541,
  10604,
  14736,
  11754,
  13928,
  14322,
  21768,
  18163,
  19414,
  22592,
  11703,
  20626,
  27983,
  11471,
  23819,
  18542,
  13901,
  11308,
  21004,
  14656,
  26542,
  39917,
  14891,
  25785,
  48725,
  14218,
  19561,
  10890,
  21709,
  27828,
  13047,
  12749,
  12570,
  13688,
  23915,
  20482,
  10161,
  40416,
  43916,
  13856,
  35211,
  19350,
  18403,
  24109,
  22103,
  24942,
  15579,
  34833,
  19254,
  24643,
  18374,
  25005,
  19537,
  17830,
  10225,
  43123,
  28484,
  21573,
  14525,
  20853,
  33814,
  25769,
  43449,
  15368,
  18411,
  24715,
  47941,
  14090,
  23341,
  28134,
  12052,
  22779,
  15720,
  18550,
  18948,
  11404,
  36505,
  46682,
  20730,
  27051,
  26825,
  10989,
  10145,
  59328,
  52564,
  11981,
  27959,
  26729,
  39087,
  36679,
  23712,
  17144,
  23317,
  26403,
  27086,
  21397,
  40635,
  48506,
  27190,
  18729,
  26294,
  24272,
  14

In [196]:
### Filter out top 1000 firms from RDC_ME df
# init an empty mask
filter_mask = pd.Series([False] * len(RDC_copy))

# iterate through the dictionary of top firms
for year, lpermno in top_firms.items():
    # update the mask to identify rows to remove
    mask_for_year = (RDC_copy['year'] == year) & (RDC_copy['LPERMNO'].isin(lpermno))
    filter_mask = filter_mask | mask_for_year

# use the inverse of the mask to filter the dataframe
RDC_copy = RDC_copy[~filter_mask]
RDC_copy

,datadate,LPERMNO,xrd,mkt_cap,year,RDC,RDC_ME
89497,1979-09-30,84794,0.000,39.864750,1979,0.0000,0.000000
135535,1979-12-31,84145,0.000,11.243248,1979,0.0000,0.000000
134879,1979-03-31,83783,0.813,63.321500,1979,1.3866,0.021898
134782,1979-01-31,83724,0.000,22.035000,1979,0.0000,0.000000
134572,1979-06-30,83620,0.375,11.461750,1979,0.9468,0.082605
...,...,...,...,...,...,...,...
169466,2021-12-31,10051,0.000,702.519370,2021,0.0000,0.000000
137910,2021-02-28,10044,0.000,32.131460,2021,0.0000,0.000000
146749,2021-09-30,10032,0.000,2507.682270,2021,0.0000,0.000000
138576,2021-12-31,10028,0.000,109.584750,2021,0.0000,0.000000


In [197]:
# split into quintiles
securities_ME_port = pd.DataFrame()
for year in RDC_copy['year'].unique():
    data = RDC_copy[RDC_copy['year']==year]
    port_xrd = data[data['RDC_ME']>0]
    data['flg_portfolio'] = np.where(data['RDC_ME']<=0,0,
        np.where((data['RDC_ME']>0)&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.2)),1,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.2))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.4)),2,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.4))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.6)),3,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.6))&(data['RDC_ME']<=port_xrd['RDC_ME'].quantile(0.8)),4,
        np.where((data['RDC_ME']>port_xrd['RDC_ME'].quantile(0.8)),5, np.nan))))))
    securities_ME_port = pd.concat([securities_ME_port,data])
print(len(securities_ME_port))
# shift forward one year to match returns
securities_ME_port['year'] = securities_ME_port['year']+1
securities_ME_port.head()

101240


,datadate,LPERMNO,xrd,mkt_cap,year,RDC,RDC_ME,flg_portfolio
89497,1979-09-30,84794,0.000,39.864750,1980,0.0000,0.000000,0.0
135535,1979-12-31,84145,0.000,11.243248,1980,0.0000,0.000000,0.0
134879,1979-03-31,83783,0.813,63.321500,1980,1.3866,0.021898,2.0
134782,1979-01-31,83724,0.000,22.035000,1980,0.0000,0.000000,0.0
134572,1979-06-30,83620,0.375,11.461750,1980,0.9468,0.082605,3.0


In [198]:
# split quintiles into their own dfs
Zero_RD_df = securities_ME_port[securities_ME_port['flg_portfolio']==0]
Q1 = securities_ME_port[securities_ME_port['flg_portfolio']==1]
Q2 = securities_ME_port[securities_ME_port['flg_portfolio']==2]
Q3 = securities_ME_port[securities_ME_port['flg_portfolio']==3]
Q4 = securities_ME_port[securities_ME_port['flg_portfolio']==4]
Q5 = securities_ME_port[securities_ME_port['flg_portfolio']==5]

In [199]:
# merge the quintiles with the returns df
Q1_rtns = pd.merge(Q1, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q2_rtns = pd.merge(Q2, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q3_rtns = pd.merge(Q3, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q4_rtns = pd.merge(Q4, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
Q5_rtns = pd.merge(Q5, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')
ZeroRD_rtns = pd.merge(Zero_RD_df, sec_mon[['LPERMNO', 'year', 'month', 'trt1m']], on=['LPERMNO', 'year'], how='left')

In [200]:
# drop NaNs and convert back to int
Q1_rtns = Q1_rtns.dropna(subset=['month'])
Q1_rtns['month'] = Q1_rtns['month'].astype(int)

Q2_rtns = Q2_rtns.dropna(subset=['month'])
Q2_rtns['month'] = Q2_rtns['month'].astype(int)

Q3_rtns = Q3_rtns.dropna(subset=['month'])
Q3_rtns['month'] = Q3_rtns['month'].astype(int)

Q4_rtns = Q4_rtns.dropna(subset=['month'])
Q4_rtns['month'] = Q4_rtns['month'].astype(int)

Q5_rtns = Q5_rtns.dropna(subset=['month'])
Q5_rtns['month'] = Q5_rtns['month'].astype(int)

ZeroRD_rtns = ZeroRD_rtns.dropna(subset=['month'])
ZeroRD_rtns['month'] = ZeroRD_rtns['month'].astype(int)

In [201]:
# drop duplicates of years and securities so we can grab just the yearly market cap
yr_mkt_cap = Q1_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
# select only necessary columns for our calculation & merge
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
# find the total market cap for each year
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
# find the ratio of each security's mkt cap to total mkt cap for that year
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
# merge the calculated weights back to the original dataframe on year and LPERMNO
Q1_rtns_all = Q1_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
# create new column for the weighted return
Q1_rtns_all['weighted_return'] = Q1_rtns_all['trt1m'] * Q1_rtns_all['weight']
Q1_rtns_all

,datadate,LPERMNO,xrd,mkt_cap,year,RDC,RDC_ME,flg_portfolio,month,trt1m,weight,weighted_return
0,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,4,68.7500,0.007956,0.546952
1,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,6,8.1818,0.007956,0.065092
2,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,5,1.8519,0.007956,0.014733
3,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,2,-2.5641,0.007956,-0.020399
4,1979-03-31,83300,0.200,26.13000,1980,0.288,0.011022,1.0,3,-14.6842,0.007956,-0.116822
...,...,...,...,...,...,...,...,...,...,...,...,...
116841,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,11,11.1171,0.009129,0.101483
116842,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,8,9.9771,0.009129,0.091077
116843,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,4,-3.4816,0.009129,-0.031782
116844,2021-09-30,10026,0.619,2916.41688,2022,1.934,0.000663,1.0,3,-4.8785,0.009129,-0.044534


In [202]:
# find value-weighted returns for the rest of the portfolios
yr_mkt_cap = Q2_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q2_rtns_all = Q2_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q2_rtns_all['weighted_return'] = Q2_rtns_all['trt1m'] * Q2_rtns_all['weight']

yr_mkt_cap = Q3_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q3_rtns_all = Q3_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q3_rtns_all['weighted_return'] = Q3_rtns_all['trt1m'] * Q3_rtns_all['weight']

yr_mkt_cap = Q4_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q4_rtns_all = Q4_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q4_rtns_all['weighted_return'] = Q4_rtns_all['trt1m'] * Q4_rtns_all['weight']

yr_mkt_cap = Q5_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
Q5_rtns_all = Q5_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
Q5_rtns_all['weighted_return'] = Q5_rtns_all['trt1m'] * Q5_rtns_all['weight']

yr_mkt_cap = ZeroRD_rtns.drop_duplicates(subset=['year', 'LPERMNO'])
yr_mkt_cap = yr_mkt_cap[['LPERMNO', 'year', 'mkt_cap']]
tot_mkt_cap = yr_mkt_cap.groupby('year')['mkt_cap'].transform('sum')
yr_mkt_cap['weight'] = yr_mkt_cap['mkt_cap'] / tot_mkt_cap
ZeroRD_rtns_all = ZeroRD_rtns.merge(yr_mkt_cap[['year', 'LPERMNO', 'weight']], on=['year', 'LPERMNO'], how='left')
ZeroRD_rtns_all['weighted_return'] = ZeroRD_rtns_all['trt1m'] * ZeroRD_rtns_all['weight']

In [203]:
# calculate total monthly returns
tot_Q1_rtns = Q1_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q2_rtns = Q2_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q3_rtns = Q3_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q4_rtns = Q4_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_Q5_rtns = Q5_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()
tot_ZeroRD_rtns = ZeroRD_rtns_all.groupby(['year', 'month'])['weighted_return'].sum().reset_index()

tot_Q1_rtns

,year,month,weighted_return
0,1980,1,11.124756
1,1980,2,-0.809626
2,1980,3,-20.678422
3,1980,4,7.441306
4,1980,5,5.884915
...,...,...,...
511,2022,8,-3.109023
512,2022,9,-11.062297
513,2022,10,9.945297
514,2022,11,-3.447407


In [204]:
# combine year and month into a dt column
tot_Q1_rtns['date'] = pd.to_datetime(tot_Q1_rtns['year'].astype(str) + '-' + tot_Q1_rtns['month'].astype(str) + '-01')
tot_Q2_rtns['date'] = pd.to_datetime(tot_Q2_rtns['year'].astype(str) + '-' + tot_Q2_rtns['month'].astype(str) + '-01')
tot_Q3_rtns['date'] = pd.to_datetime(tot_Q3_rtns['year'].astype(str) + '-' + tot_Q3_rtns['month'].astype(str) + '-01')
tot_Q4_rtns['date'] = pd.to_datetime(tot_Q4_rtns['year'].astype(str) + '-' + tot_Q4_rtns['month'].astype(str) + '-01')
tot_Q5_rtns['date'] = pd.to_datetime(tot_Q5_rtns['year'].astype(str) + '-' + tot_Q5_rtns['month'].astype(str) + '-01')
tot_ZeroRD_rtns['date'] = pd.to_datetime(tot_ZeroRD_rtns['year'].astype(str) + '-' + tot_ZeroRD_rtns['month'].astype(str) + '-01')

In [205]:
# Full sample period (1981.07 – 2022.12)
start_date = '1981-07-01'
end_date = '2022-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.42 0.876 1.026 1.245 1.748 0.852


In [206]:
# Extended period (2013.01 – 2022.12)
start_date = '2013-01-01'
end_date = '2022-12-31'

# calculate the average monthly return for the full sample period
Q1_rtns_FULL = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]['weighted_return'].mean()
Q2_rtns_FULL = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]['weighted_return'].mean()
Q3_rtns_FULL = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]['weighted_return'].mean()
Q4_rtns_FULL = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]['weighted_return'].mean()
Q5_rtns_FULL = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]['weighted_return'].mean()
ZeroRD_rtns_FULL = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]['weighted_return'].mean()

print(round(Q1_rtns_FULL, 3), round(Q2_rtns_FULL, 3), round(Q3_rtns_FULL, 3), round(Q4_rtns_FULL, 3), round(Q5_rtns_FULL, 3), round(ZeroRD_rtns_FULL, 3))

0.497 0.919 1.185 1.153 1.897 0.804


In [207]:
# Q5 = high R&D      Q1 = low R&D
longshort = pd.merge(tot_Q5_rtns, tot_Q1_rtns, on=['year', 'month'], how='left', suffixes=['_high_RD', '_low_RD'])
longshort['long_short'] = longshort['weighted_return_high_RD'] - longshort['weighted_return_low_RD']
longshort['date'] = longshort['date_high_RD']
longshort['month'] = longshort['date'].dt.month
longshort['year'] = longshort['date'].dt.year
longshort = longshort[['date','year','month','long_short']]
longshort

,date,year,month,long_short
0,1980-01-01,1980,1,-2.755473
1,1980-02-01,1980,2,-1.618327
2,1980-03-01,1980,3,3.410739
3,1980-04-01,1980,4,-4.846813
4,1980-05-01,1980,5,1.739044
...,...,...,...,...
511,2022-08-01,2022,8,15.068120
512,2022-09-01,2022,9,-1.034403
513,2022-10-01,2022,10,-4.317110
514,2022-11-01,2022,11,1.329962


In [208]:
ls_idx = pd.merge(longshort, ff_df, on=['year', 'month'], how='left')
start_date = '1981-07-01'
end_date = '2022-12-31'
ls_idx = ls_idx[(ls_idx['date'] >= start_date) & (ls_idx['date'] <= end_date)]
ls_idx

,date,year,month,long_short,Mkt-RF,SMB,HML,RF
18,1981-07-01,1981,7,0.497682,-1.54,-2.19,-0.50,1.24
19,1981-08-01,1981,8,2.794515,-7.04,-1.95,4.76,1.28
20,1981-09-01,1981,9,4.272547,-7.17,-2.65,5.17,1.24
21,1981-10-01,1981,10,-1.678006,4.92,2.23,-4.21,1.21
22,1981-11-01,1981,11,1.867983,3.36,-1.03,1.83,1.07
...,...,...,...,...,...,...,...,...
511,2022-08-01,2022,8,15.068120,-3.77,1.40,0.29,0.19
512,2022-09-01,2022,9,-1.034403,-9.35,-0.81,0.05,0.19
513,2022-10-01,2022,10,-4.317110,7.83,0.06,8.01,0.23
514,2022-11-01,2022,11,1.329962,4.60,-3.52,1.38,0.29


In [209]:
### CAPM alpha
# declare independent and dependent variable from dataframe
X = ls_idx['Mkt-RF']
y = ls_idx['long_short']
# add a constant to the our independent variable to allow for non-zero intercept
X2 = sm.add_constant(X)

# create the OLS regression model and fit it
lin_reg = sm.OLS(y, X2).fit()
# print a summary
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     12.74
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           0.000393
Time:                        19:11:46   Log-Likelihood:                -1468.0
No. Observations:                 498   AIC:                             2940.
Df Residuals:                     496   BIC:                             2948.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2181      0.209      5.817      0.0

In [210]:
### Fama-French 3-factor alpha
X = ls_idx[['Mkt-RF', 'SMB', 'HML']]
y = ls_idx['long_short']
X2 = sm.add_constant(X)
lin_reg = sm.OLS(y, X2).fit()
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:             long_short   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     14.34
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           5.71e-09
Time:                        19:11:46   Log-Likelihood:                -1453.5
No. Observations:                 498   AIC:                             2915.
Df Residuals:                     494   BIC:                             2932.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2449      0.205      6.068      0.0

In [211]:
### Sharpe Ratio
sr = ls_idx['long_short'].mean()/ls_idx['long_short'].std()
sr*np.sqrt(12)

0.9839755973989105

In [212]:
### grab dataframe for final df
start_date = '1981-07-01'
end_date = '2022-12-31'

# filter the returns to the sample period
Q1_rtns_df = tot_Q1_rtns[(tot_Q1_rtns['date'] >= start_date) & (tot_Q1_rtns['date'] <= end_date)]
Q2_rtns_df = tot_Q2_rtns[(tot_Q2_rtns['date'] >= start_date) & (tot_Q2_rtns['date'] <= end_date)]
Q3_rtns_df = tot_Q3_rtns[(tot_Q3_rtns['date'] >= start_date) & (tot_Q3_rtns['date'] <= end_date)]
Q4_rtns_df = tot_Q4_rtns[(tot_Q4_rtns['date'] >= start_date) & (tot_Q4_rtns['date'] <= end_date)]
Q5_rtns_df = tot_Q5_rtns[(tot_Q5_rtns['date'] >= start_date) & (tot_Q5_rtns['date'] <= end_date)]
ZeroRD_rtns_df = tot_ZeroRD_rtns[(tot_ZeroRD_rtns['date'] >= start_date) & (tot_ZeroRD_rtns['date'] <= end_date)]

# merge all the returns into one df
step4df = pd.merge(Q1_rtns_df[['weighted_return', 'date']], Q2_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q2'])
step4df = pd.merge(step4df, Q3_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q3'])
step4df = pd.merge(step4df, Q4_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q4'])
step4df = pd.merge(step4df, Q5_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_Q5'])
step4df = pd.merge(step4df, ZeroRD_rtns_df[['weighted_return', 'date']], on='date', how='left', suffixes=['', '_NoRD'])
step4df = pd.merge(step4df, ls_idx[['long_short', 'date']], on='date', how='left')
# rename then drop the Q1 weighted return
step4df['weighted_return_Q1'] = step4df['weighted_return']
step4df.drop('weighted_return', axis=1, inplace=True)
step4df

,date,weighted_return_Q2,weighted_return_Q3,weighted_return_Q4,weighted_return_Q5,weighted_return_NoRD,long_short,weighted_return_Q1
0,1981-07-01,-5.281838,-2.596946,-2.321707,-2.818071,-1.722863,0.497682,-3.315753
1,1981-08-01,-10.931691,-9.291505,-11.794023,-9.179095,-7.033458,2.794515,-11.973611
2,1981-09-01,-9.111137,-6.036903,-9.531048,-8.450143,-9.149231,4.272547,-12.722690
3,1981-10-01,10.615221,9.892661,6.996891,9.174347,7.511510,-1.678006,10.852353
4,1981-11-01,0.293262,4.959184,1.452266,2.617675,1.484366,1.867983,0.749692
...,...,...,...,...,...,...,...,...
493,2022-08-01,-0.261868,1.019496,3.129928,11.959097,-0.165893,15.068120,-3.109023
494,2022-09-01,-10.459339,-9.891858,-7.257909,-12.096700,-11.702807,-1.034403,-11.062297
495,2022-10-01,10.418972,9.275078,3.481193,5.628187,11.878891,-4.317110,9.945297
496,2022-11-01,-4.867598,2.513726,-0.283410,-2.117444,1.134669,1.329962,-3.447407


In [213]:
### combine dfs from each part into one final df
final_df = pd.merge(step2df, step3df, on='date', how='left', suffixes=['_EW', '_VW'])
final_df = pd.merge(final_df, step4df, on='date', how='left')
final_df.rename(columns={'weighted_return_Q2': 'weighted_return_Q2_SMID', 'weighted_return_Q3': 'weighted_return_Q3_SMID', 'weighted_return_Q4': 'weighted_return_Q4_SMID', 'weighted_return_Q5': 'weighted_return_Q5_SMID', 'weighted_return_Q1': 'weighted_return_Q1_SMID', 'weighted_return_NoRD': 'weighted_return_NoRD_SMID', 'long_short': 'long_short_SMID'}, inplace=True)
final_df

,date,weighted_return_Q2_EW,weighted_return_Q3_EW,weighted_return_Q4_EW,weighted_return_Q5_EW,weighted_return_NoRD_EW,weighted_return_Q1_EW,long_short_EW,weighted_return_Q2_VW,weighted_return_Q3_VW,...,weighted_return_NoRD_VW,long_short_VW,weighted_return_Q1_VW,weighted_return_Q2_SMID,weighted_return_Q3_SMID,weighted_return_Q4_SMID,weighted_return_Q5_SMID,weighted_return_NoRD_SMID,long_short_SMID,weighted_return_Q1_SMID
0,1981-07-01,-2.799800,-3.457001,-3.606291,-4.290787,-1.309947,-1.553359,-2.737428,3.212585,-2.897262,...,1.066085,-14.628404,8.417815,-5.281838,-2.596946,-2.321707,-2.818071,-1.722863,0.497682,-3.315753
1,1981-08-01,-8.570050,-8.670321,-10.652510,-9.499001,-6.909227,-9.112646,-0.386355,-3.777345,-7.103646,...,-5.588584,-0.845318,-7.445599,-10.931691,-9.291505,-11.794023,-9.179095,-7.033458,2.794515,-11.973611
2,1981-09-01,-9.134037,-6.536604,-7.867166,-7.552083,-8.629938,-11.351025,3.798942,-6.033636,-6.255274,...,-6.989747,7.144450,-11.332518,-9.111137,-6.036903,-9.531048,-8.450143,-9.149231,4.272547,-12.722690
3,1981-10-01,9.297802,8.675598,7.015914,6.444521,7.552285,11.211338,-4.766817,6.206295,6.414784,...,7.505173,-11.189724,9.776525,10.615221,9.892661,6.996891,9.174347,7.511510,-1.678006,10.852353
4,1981-11-01,0.127447,4.699210,1.494010,2.537728,2.295251,1.763461,0.774267,4.728657,4.806934,...,4.037957,-4.075303,6.771236,0.293262,4.959184,1.452266,2.617675,1.484366,1.867983,0.749692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,2022-08-01,-3.811503,-1.377376,1.296049,5.227380,0.022568,-1.852596,7.079976,-5.610173,-2.828746,...,-2.232595,6.547610,-3.992962,-0.261868,1.019496,3.129928,11.959097,-0.165893,15.068120,-3.109023
494,2022-09-01,-11.264344,-11.418521,-10.947752,-14.819228,-12.201691,-11.184550,-3.634678,-11.104407,-11.546655,...,-9.452919,0.176981,-8.600926,-10.459339,-9.891858,-7.257909,-12.096700,-11.702807,-1.034403,-11.062297
495,2022-10-01,8.146449,5.740109,4.810552,1.081895,10.038750,5.439642,-4.357747,5.511529,-0.683491,...,8.841528,8.627664,5.261553,10.418972,9.275078,3.481193,5.628187,11.878891,-4.317110,9.945297
496,2022-11-01,-0.568152,0.196352,-0.152652,-3.669819,1.933884,-0.056066,-3.613753,5.510540,7.149928,...,6.882075,3.529738,1.000828,-4.867598,2.513726,-0.283410,-2.117444,1.134669,1.329962,-3.447407


In [214]:
# export final dataframe to CSV
final_df.to_csv('Assignment1_final_dataframe.csv', index=False)